In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sodapy import Socrata #need to use pip install sodapy to get this, using it to get data from the CDC website
import os

data_prefix = 'Data/'
output_prefix_01 = '01_data_processing.ipynb.output/'

## Getting Data

In [2]:
#make a Data folder if one doesn't exist
if not os.path.exists(data_prefix):
    os.makedirs(data_prefix)

#check if the data files exist, otherwise pull from the CDC website
if not os.path.exists(data_prefix + 'covid_rates.csv'):
    print('Downloading data from CDC website')
    ## pulling data from CDC website using Socrata API
    # Create a client object
    client = Socrata("data.cdc.gov", None)

    # pull rsv, covid, and flu rates data https://data.cdc.gov/Public-Health-Surveillance/Rates-of-Laboratory-Confirmed-RSV-COVID-19-and-Flu/kvib-3txy/about_data
    covid_rates_results = client.get("kvib-3txy",limit=47333)
    # Convert to pandas DataFrame
    covid_rates_df = pd.DataFrame.from_records(covid_rates_results)
    #pull youth obesity data https://data.cdc.gov/Nutrition-Physical-Activity-and-Obesity/Nutrition-Physical-Activity-and-Obesity-Youth-Risk/vba9-s8jp/about_data
    youth_results = client.get("vba9-s8jp", limit=50092)
    # Convert to pandas DataFrame
    youth_df = pd.DataFrame.from_records(youth_results)
    #pull adult obesity data https://chronicdata.cdc.gov/Nutrition-Physical-Activity-and-Obesity/Nutrition-Physical-Activity-and-Obesity-Behavioral/hn4x-zwk7/about_data
    adult_results = client.get("hn4x-zwk7", limit=104272)
    # Convert to pandas DataFrame
    adult_df = pd.DataFrame.from_records(adult_results)

    #download the data into the data folder
    covid_rates_df.to_csv(data_prefix + 'covid_rates.csv', index=False)
    youth_df.to_csv(data_prefix + 'youth_obesity.csv', index=False)
    adult_df.to_csv(data_prefix + 'adult_obesity.csv', index=False)
else:
    print('Data already downloaded')
    #read in the data
    covid_rates_df = pd.read_csv(data_prefix + 'covid_rates.csv')
    youth_df = pd.read_csv(data_prefix + 'youth_obesity.csv')
    adult_df = pd.read_csv(data_prefix + 'adult_obesity.csv')


Data already downloaded


In [3]:
print(youth_df.columns)
print(adult_df.columns)

Index(['yearstart', 'yearend', 'locationabbr', 'locationdesc', 'datasource',
       'class', 'topic', 'question', 'data_value_type', 'data_value',
       'data_value_alt', 'low_confidence_limit', 'high_confidence_limit',
       'sample_size', 'race_ethnicity', 'geolocation', 'classid', 'topicid',
       'questionid', 'datavaluetypeid', 'locationid',
       'stratificationcategory1', 'stratification1',
       'stratificationcategoryid1', 'stratificationid1', 'total', 'grade',
       'sex', 'data_value_footnote_symbol', 'data_value_footnote'],
      dtype='object')
Index(['yearstart', 'yearend', 'locationabbr', 'locationdesc', 'datasource',
       'class', 'topic', 'question', 'data_value_unit', 'data_value_type',
       'data_value', 'data_value_alt', 'low_confidence_limit',
       'high_confidence_limit', 'sample_size', 'race_ethnicity', 'geolocation',
       'classid', 'topicid', 'questionid', 'datavaluetypeid', 'locationid',
       'stratificationcategory1', 'stratification1',
      

## Data Processing

In [4]:
# #fixing values in the 'Race/Ethnicity' column
# #there are two years where some of the answers are not capitalized but should be part of the same category
# youth_df['Race/Ethnicity'] = youth_df['Race/Ethnicity'].replace('non-Hispanic black', 'Non-Hispanic Black')
# youth_df['Race/Ethnicity'] = youth_df['Race/Ethnicity'].replace('non-Hispanic white', 'Non-Hispanic White')

In [5]:
#transforming youth dataframe
#pivotting the dataframe so that each row for a given year and location, and each column is the average value for different demographics for that metric
youth_df_very_wide = youth_df.pivot_table(index=['yearstart','locationabbr','geolocation'],columns=['class',"stratificationid1"],values="data_value")
#change the index to columns
youth_df_very_wide.reset_index(inplace=True)
#smoosh the column index
youth_df_very_wide.columns = ['_'.join(col).strip() for col in youth_df_very_wide.columns.values]
#replace whitespace with underscores
youth_df_very_wide.columns = youth_df_very_wide.columns.str.replace(' ', '')
#remove trailing underscores
youth_df_very_wide.columns = youth_df_very_wide.columns.str.rstrip('_')
#add a prefix 'Youth' to all columns except for the first three
youth_df_very_wide.columns = ['youth_' + col if col not in ['yearstart','locationabbr','geolocation'] else col for col in youth_df_very_wide.columns]

#transform adult dataframe
#pivotting the dataframe so that each row for a given year and location, and each column is the average value for different demographics for that metric
adult_df_very_wide = adult_df.pivot_table(index=['yearstart','locationabbr','geolocation'],columns=['class',"stratificationid1"],values="data_value")
#change the index to columns
adult_df_very_wide.reset_index(inplace=True)
#smoosh the column index
adult_df_very_wide.columns = ['_'.join(col).strip() for col in adult_df_very_wide.columns.values]
#replace whitespace with underscores
adult_df_very_wide.columns = adult_df_very_wide.columns.str.replace(' ', '')
#remove trailing underscores
adult_df_very_wide.columns = adult_df_very_wide.columns.str.rstrip('_')
#add a prefix 'Adult' to all columns except for the first three
adult_df_very_wide.columns = ['adult_' + col if col not in ['yearstart','locationabbr','geolocation'] else col for col in adult_df_very_wide.columns]

#merge the two dataframes on YearStart and LocationAbbr
full_df_very_wide = pd.merge(youth_df_very_wide, adult_df_very_wide, on=['yearstart','locationabbr','geolocation'], how='outer')

full_df_very_wide.head()

,yearstart,locationabbr,geolocation,youth_FruitsandVegetables_FEMALE,youth_FruitsandVegetables_GRADE09,youth_FruitsandVegetables_GRADE10,youth_FruitsandVegetables_GRADE11,youth_FruitsandVegetables_GRADE12,youth_FruitsandVegetables_MALE,youth_FruitsandVegetables_OVERALL,...,adult_PhysicalActivity_MALE,adult_PhysicalActivity_OVERALL,adult_PhysicalActivity_RACE2PLUS,adult_PhysicalActivity_RACEASN,adult_PhysicalActivity_RACEBLK,adult_PhysicalActivity_RACEHIS,adult_PhysicalActivity_RACEHPI,adult_PhysicalActivity_RACENAA,adult_PhysicalActivity_RACEOTH,adult_PhysicalActivity_RACEWHT
0,2001,AL,"{'latitude': '32.840571122', 'longitude': '-86...",49.95,52.75,50.80,50.05,44.30,49.75,49.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2001,AR,"{'latitude': '34.748650124', 'longitude': '-92...",43.25,41.35,43.90,40.95,44.35,42.05,42.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2001,DE,"{'latitude': '39.008830667', 'longitude': '-75...",33.50,34.10,31.95,34.55,30.85,32.30,32.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2001,FL,"{'latitude': '28.932040377', 'longitude': '-81...",41.05,40.80,40.25,40.20,44.65,41.60,41.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2001,ID,"{'latitude': '43.682630005', 'longitude': '-11...",34.70,35.50,37.75,33.05,38.00,37.30,36.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#if output folder doesn't exist, make it
if not os.path.exists(output_prefix_01):
    os.makedirs(output_prefix_01)
#outputting the data to a csv file
full_df_very_wide.to_csv(output_prefix_01+'metadata_very_wide.csv', index=False)